<a href="https://colab.research.google.com/github/johyunkang/python_stat/blob/main/chapter11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11 통계적 가설 검정

## 11.1 통계적 가설검정

In [1]:
from scipy import stats
import pandas as pd
import numpy as np

# df = pd.read_csv('감자튀김.csv')
# sample = np.array(df['무게'])

potato_sample = [122.02, 131.73, 130.6, 131.82, 132.05, 126.12, 124.43,
          132.89, 122.79, 129.95, 126.14, 134.45, 127.64, 125.68]

mu = 130
var = 9
n = 14
# rv = stats.norm(130, np.sqrt(9/14)) # 감튀 평균:130, 분산:9, 표본: 14개
rv = stats.norm(mu, np.sqrt(var/n)) # 감튀 평균:130, 분산:9, 표본: 14개

alpha = 0.05
rv = stats.norm()
print('임곗값:', rv.isf(1 - alpha)) # isf (Inverse Survival Function) : 임곗값 산출
# > -1.645

potato_mean = np.mean(potato_sample)
print('감자샘플 평균 무게:', potato_mean)
# > 128.451

Z = (potato_mean - mu) / np.sqrt(var/n) # 검정통계량 값 Z 를 구함
print('검정통계량(Z):', Z)
# > -1.932

# p-value 는 누적분포함수(cdf)를 이용해 산출 가능
p_val = rv.cdf(Z)
print('p_value:', p_val)
# > 0.027

# p-value 가 유의수준(a) 0.05 보다 작으므로 귀무가설을 기각한다.


임곗값: -1.6448536269514722
감자샘플 평균 무게: 128.4507142857143
검정통계량(Z): -1.932298779026813
p_value: 0.026661319523126635


$Z = \dfrac{(\bar{X} - 130)} { \sqrt{\dfrac{9} {14}}}$

p-value 를 기준으로 수행하는 가설검정의 흐름

## 11.2 기본적인 가설검정

### 11.2.1 정규분포의 모평균에 대한 검정 : 모분산을 알고 있는 경

모평균에 대한 검정이란 **모평균이 어떤 값 $\mu_0$이 아니라고 주장**하기 위한 검정


검정통계량으로 Z를 사용 (양측검정)

$Z = \dfrac {(\bar{X} - \mu_0)} {\sqrt{\dfrac {\sigma^2} {n}}} $

- 귀무가설을 기각 : $Z < z_{1 - \alpha / 2} \quad or \quad z_{a/2} < Z$
- 귀무가설을 채택 : $z_{1-\alpha/2} \leq  Z \leq z_{\alpha/2}$


위 식을 아래 파이썬으로 구현

In [2]:
def potato_mean_test(sample, mean0, p_var, alpha=0.05) :
    import numpy as np
    from scipy import stats

    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1 - alpha)

    z = (s_mean - mean0) / np.sqrt(p_var / n)

    if interval[0] <= z <= interval[1] :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 채택')
    else :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 기각')
    
    if z < 0 :
        p_value = rv.cdf(z) * 2 # 양측검정이라 * 2 해줌
    else :
        p_value = (1- rv.cdf(z)) * 2 # 양측검정이라 * 2 해줌

    print(f'p-value는 {p_value:.3f} 입니다.')

In [3]:
potato_sample = [122.02, 131.73, 130.6, 131.82, 132.05, 126.12, 124.43,
          132.89, 122.79, 129.95, 126.14, 134.45, 127.64, 125.68]

mu = 130
var = 9
potato_mean_test(potato_sample, mu, var)

유의수준 0.050에서 귀무가설을 채택
p-value는 0.053 입니다.


### 11.2.2 정규분포의 모분산에 대한 검정

모분산에 대한 검정은 **모분산이 어떤 값 $\sigma_0^2$ 이 아닌 것을 주장하기 위한 검정**입니다.

$N(\mu, \sigma^2)$ 이라고 함. 모분산 $\sigma^2$에 관한 유의수준 $\alpha$의 양측검정

- 귀무가설 : $\sigma^2 = \sigma_0^2$
- 대림가설 : $\sigma^2 \neq \sigma_0^2$

검정통계량으로 $Y = \dfrac {(n-1)s^2} {\sigma_0^2}$ 을 사용

- 귀무가설을 채택 : $X_{1-\alpha/2}^2(n-1)  \leq Y \leq X_{\alpha/2}^2 (n-1)$
- 귀무가설을 기각 : $Y \lt X_{1-\alpha/2}^2(n-1) \quad or \quad X_{\alpha/2}^2 (n-1) \lt Y$

In [4]:
def potato_var_test(sample, var0, alpha=0.05) :
    import numpy as np
    from scipy import stats

    u_var = np.var(sample, ddof=1) # ddof=1 : 불편분산 (n-1 사용), ddof=0 : 표본분산 (n 사용)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1 - alpha)

    y = (n - 1) * u_var / var0

    if interval[0] <= y <= interval[1] :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 채택합니다')
    else :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 기각합니다')
    
    if y < rv.isf(0.5) : # isf (Inverse Survival Function)
        p = rv.cdf(y) * 2 # 양측검정이라 * 2 
    else :
        p = (1 - rv.cdf(y)) * 2 # 양측검정이라 * 2
    
    print(f'p-value는 {p:.3f} 입니다.')
    print('Interval 값은 :', interval)
    print('검정통계량 Y : ', y)

In [5]:
potato_var_test(potato_sample, var)

유의수준 0.050에서 귀무가설을 채택합니다
p-value는 0.085 입니다.
Interval 값은 : (5.008750511810331, 24.735604884931547)
검정통계량 Y :  22.921810317460263


### 11.2.3 정규분포의 모평균에 대한 검정 : 모분산을 모르는 경우

**모분산을 알지 못하는 상황**에서 정규분포의 모평균에 대한 검정을 **1표본 t검정 (1-sample t-test)**이라고 부르고, **t 검정통계랑** 이라고 하는

 $t = \dfrac {(\bar{X} - \mu_0)} {\sqrt{\dfrac {s^2} {n}}}$ 
 
 을 검정통계량으로 사용함

이 t 검정통계량은 자유도가 n-1 인 t분포를 따름

모평균 &mu;에 관한 유의수준 &alpha;의 **양측검정**

- 귀무가설 : $\mu = \mu_0$
- 대림가설 : $\mu \neq \mu_0$

검정통계량으로 $t = \dfrac {(\bar{X} - \mu_0)} {\sqrt{\dfrac {s^2} {n}}}$ 를 사용하여 

- 귀무가설 기각 : $t < t_{1- a/2}(n-1) \quad OR \quad t_{a/2} \lt t $
- 귀무가설 채택 : $t_{1-a/2}(n-1) \quad \le \quad t \quad \le \quad t_{a/2}(n-1)$




위 식을 아래 함수 형식으로 구현



In [6]:
def potato_t_test(sample, mean0, alpha=0.05) :
    import numpy as np
    from scipy import stats

    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1) # ddof=1 : 불편분산 (n-1 사용), ddof=0 : 표본분산 (n 사용)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1 - alpha)

    t = (s_mean - mean0) / np.sqrt(u_var / n)

    if interval[0] <= t <= interval[1] :
        print(f't 값이 {t:.3f} 이기에 유의수준 {alpha}에서 귀무가설을 채택')
    else :
        print(f't 값이 {t:.3f} 이기에 유의수준 {alpha}에서 귀무가설을 기각')
    print('\nInterval:', interval)

    if t < 0 :
        p = rv.cdf(t) * 2 # 양측검정이기에 * 2
    else :
        p = (1 - rv.cdf(t)) * 2
    
    print(f'p-value 는 {p:.4f} 입니다.')


potato_t_test(potato_sample, 130, 0.01)

t 값이 -1.455 이기에 유의수준 0.01에서 귀무가설을 채택

Interval: (-3.012275838207184, 3.012275838207184)
p-value 는 0.1693 입니다.


1표본 t 검정은 `scipy.stats`에서 `ttest_1samp` 함수로 구현되어 있고

반환값은 `t검정통계량`과 `p-value` 입니다.

In [7]:
t, p_val = stats.ttest_1samp(potato_sample, 130, alternative='two-sided')
print('검정통계량 t:', t)
print('p-value:', p_val)
# print('df는 ', df, ' 입니다.')
stats.ttest_1samp(potato_sample, 130, alternative='two-sided')

검정통계량 t: -1.4551960206404198
p-value: 0.16933464230414275


Ttest_1sampResult(statistic=-1.4551960206404198, pvalue=0.16933464230414275)

위 `ttest_1samp` 의 파라미터 중 `alternative` 옵션에 관해서 알아보자
- two_sided : 양측검정에 사용. 표본의 기본 분포의 평균이 모평균과 다릅니다.
- less : 표본의 기본 분포의 평균이 모집단 평균보다 작습니다.
- greater : 표본의 기본 분포의 평균이 모집단 평균보다 큽니다.



## 11.3 2표본 문제에 관한 가설검정

**2표본 문제(two sample problem)** 

**평균값 차이에 대한 검정**을 아래와 같은 형태로 진행한다.



|          | 정규분포를 가정할 수 있음 | 정규분포를 가정할 수 없음 |
| -------- | ------------------------- | ------------------------- |
| 대응표본 | 대응비교 t 검정           | 윌콕슨의 부호순위검정     |
| 독립표본 | 독립비교 t 검정           | 만.위트니의 U 검정        |



- 대응표본 : 두 데이터에서 서로 대응하는 동일한 개체에 대해 각각 다른 조건으로 측정한 것을 말함
    - 예) 피검자에게 약을 투여하기 전후에 측정한 혈압
- 독립표본 : 두 데이터에서 개체가 다른 데이터로 되어 있는 독립표본
    - 예) A조 학생의 시험 점수와 B조 학생의 시험 점수는 서로 다른 학생의 시험 점수를 비교하는데 쓰이므로 독립표본

### 11.3.1 대응비교 t 검정

대응비교 t 검정 (paired t-test) : 대응하는 데이터가 있고, 정규분포를 가정할 수 있는 경우의 **평균값 차이에 대한 검정**


샘플
> A학생은 친구 20명에게 1주일간 근력 운동을 하게 하고, 운동 전후에 집중력을 측정하는 테스트를 받게 했음. 근력운동이 집중력 테스트에 유의한 차이를 내는지 확인?

In [8]:
import pandas as pd

FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/누구나파이썬통계분석/data/ch11_training_rel.csv'
df = pd.read_csv(FILE_PATH)
print('shape:', df.shape)
display(df.head())

shape: (20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


근력운동이 집중력을 향상시키는 효과 여부는

근력운동 전후의 집중력테스트 평균점수를 비교하면 됨

- 귀무가설 : $\mu_{after} - \mu_{before} = 0$
- 대립가설 : $\mu_{after} - \mu_{before} \neq 0$

In [9]:
df['차'] = df['후'] - df['전']
display(df.head())

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


근력운동이 집중력 테스트에 끼치는 영향이 없다면, 그 차이는 임의로 분산되어 평균이 0인 분포가 될 것임.

- 귀무가설 : $\mu_{diff} = 0$
- 대립가설 : $\mu_{diff} \neq 0$

더 나아가 그 차이가 각각 독립이고 동일한 정규분포를 따르고 있다고 가정할 수 있으면, 이 검정은 모분산을 모르는 경우의 정규분포의 모분산에 대한 검정, 즉 **1표본 t검정**으로 귀착될 수 있음

1표본 t검정은 `scipy.stats`의 `ttest_1samp` 함수로 계산 가능

In [12]:
from scipy import stats

t, p = stats.ttest_1samp(df['차'], 0)
print('p-value:', p)
print('p-value 가 유의수준 5% 보다 작기 때문에 귀무가설은 기각되었음')
print('근력운동이 집중력에 유의한 차이를 가져오는 것 같음')

p-value: 0.04004419061842953
p-value 가 유의수준 5% 보다 작기 때문에 귀무가설은 기각되었음
근력운동이 집중력에 유의한 차이를 가져오는 것 같음


위에서는 `diff(차)`에 의해 가설검정을 수행했지만, `ttest_rel` 함수를 사용하면 `before`와 `after`의 데이터로 동일한 검정을 수행가능

In [13]:
t, p = stats.ttest_rel(df['후'], df['전'])
print('p-value:', p)
print('p-value 가 유의수준 5% 보다 작기 때문에 귀무가설은 기각되었음')

p-value: 0.04004419061842953
p-value 가 유의수준 5% 보다 작기 때문에 귀무가설은 기각되었음


### 11.3.2 독립비교 t 검정

독립비교 t 검정 (independent t-test) : 대응하는 데이터가 없고 **독립된 2표본 모집단**에서 **정규분포**를 가정할 수 있는 경우 **평균값 차이**에 대한 검정

샘플
> A학급에는 인문계열 학생이 많고, B학급에는 체육계열 학생이 많음. B학급은 평소에도 근력운동을 하고 있음. A학급과 B학급의 집중력 테스트의 평균에서 차이가 날까?

In [16]:
df_ind = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/누구나파이썬통계분석/data/ch11_training_ind.csv')
print('shape:', df_ind.shape)
display(df_ind.head())

shape: (20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


근력운동이 집중력 향상하는 효과 여부는 A, B 학급의 집중력 테스트에서 평균점수를 비교하면 됨

- A학급 평균 : &mu;1
- B학급 평균 : &mu;2

- 귀무가설 : $\mu_1 - \mu_2 = 0$
- 대립가설 : $\mu_1 - \mu_2 \neq 0$

독립표본이므로 차이를 구해도 의미가 없음. 각 학급의 표본이 별개의 모집단에서 추출된 것이라서..

$t = \dfrac {(\bar{X} - \bar{Y}) - (\mu_1 - \mu_2)} {\sqrt{\dfrac {s_1^2} {n_1} + \dfrac {s_2^2} {n_2}}}$

위 t의 자유도(df)가

$df = \dfrac {\left(\dfrac{s_1^2} {n_1} + \dfrac {s_2^2} {n_2} \right)^2 } { \dfrac {s_1^4} {n_1^2(n_1 - 1)} + \dfrac {s_2^4} {n_2^2(n_2 - 1)}}$


인 t 분포를 따름. 이것을 `웰치의 방법` 이라고 함.

`stats.ttest_ind` 함수를 사용하면 간단히 계산 가능.

`equal_var = False`를 지정하면 `웰치의 방법으로 계산`

In [18]:
# equal_var=False : 웰치의 방법 (2표본 독립비교 t검정)
t, p = stats.ttest_ind(df_ind['A'], df_ind['B'], equal_var=False)
print('p-value:', p)
print('p-value 가 유의수준 보다 크므로, 귀무가설을 채택')
print('따라서, A학급과 B학급의 평균차이에 유의한 차이가 있다고 말할수 없다는 결론을 내림')

p-value: 0.08695731107259361
p-value 가 유의수준 보다 크므로, 귀무가설을 채택
따라서, A학급과 B학급의 평균차이에 유의한 차이가 있다고 말할수 없다는 결론을 내림


### 11.3.3 윌콕슨의 부호순위검정

윌콕슨의 부호순위검정 (Wilcoxon signed-rank test) : **대응표본**에서 차이에 **정규분포를 가정할 수 없는** 경우, **중앙값 차이에 대한 검정**

**평균이 아닌, 중앙값 차이**에 대한 검정인 것에 주의!!

`wilcoxon` 함수의 인수에 2표본 데이터를 넣어도, 차이의 데이터를 넣어도 문제는 없음

In [26]:
t, p = stats.wilcoxon(df['전'], df['후'])
print('p-value:', f'{p:.4f}')
print('p-value 가 유의수준 보다 작으므로 귀무가설을 기각합니다.')

print('\n\n')
t, p = stats.wilcoxon(df['후'], df['전'])
print('p-value:', f'{p:.4f}')
print('p-value 가 유의수준 보다 작으므로 귀무가설을 기각합니다.')

print('\n\n')
t, p = stats.wilcoxon(df['후'] -  df['전'])
print('p-value:', f'{p:.4f}')
print('p-value 가 유의수준 보다 작으므로 귀무가설을 기각합니다.')

p-value: 0.0362
p-value 가 유의수준 보다 작으므로 귀무가설을 기각합니다.



p-value: 0.0400
p-value 가 유의수준 보다 작으므로 귀무가설을 기각합니다.



p-value: 0.0400
p-value 가 유의수준 보다 작으므로 귀무가설을 기각합니다.


### 11.3.4 만.위트니의 U 검정

만.위트니의 U 검정 (Mann-Whitney rank test) : **대응데이터가 없는** 2표본 모집단에 **정규분포를 가정할 수 없는** 경우, **중앙값의 차이**에 대한 검정

`scipy.stats`에서는 U 검정을 `mannwhitneyu` 함수로 실행 가능

In [27]:
u, p = stats.mannwhitneyu(df_ind['A'], df_ind['B'], alternative='two-sided')

print('p-value:', f'{p:.4f}')
print('p-value 가 유의수준 보다 크므로 귀무가설을 채택합니다.')

p-value: 0.0595
p-value 가 유의수준 보다 크므로 귀무가설을 채택합니다.


### 11.3.5 카이제곱검정

**독립성 검정(test for independence)** : 두 변수 X와 Y에 관해서
 - 귀무가설 : 'X와 Y가 독립이다'
 - 대립가설 : 'X와 Y가 독립이 아니다'


독립성 검정에는 카이제곱분포가 사용되기 때문에 `카이제곱검정 (chi-square test)` 라고도 부름.

예제

> A, B 광고가 만들어 졌고, 광고를 내보냈을 때 구입비율에 유의한 차이가 있는지 확인하려면 어떻게 하면 좋을까요?

In [28]:
ad_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/누구나파이썬통계분석/data/ch11_ad.csv')
n = len(ad_df)
print('n:', n)
print('shape:', ad_df.shape)
display(ad_df.head())

n: 1000
shape: (1000, 2)


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


위 데이터만 가지고는 눈에 들어오지 않기 때문에 `교차집계표(cross table)`을 작성. **분할표** 라고도 부름.

`pandas`의 `crosstab` 함수로 작성 가능

In [29]:
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
display(ad_cross)

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [31]:
## 각 광고를 통한 구입 비율
display( ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])  * 100)

광고
A    12.25
B     8.50
dtype: float64

A의 구입비율이 12.25% 로 B 의 8.5%에 비해 높음.

이것은 유의한 차이일까? 

카이제곱검정을 수행하려면 몇 가지 준비가 필요
- 구입한 사람의 합계
- 상품을 구입하지 않은 사람의 합계
- 광고 A를 본 사람의 합계
- 광고 B를 본 사람의 합계

In [34]:
n_not, n_yes = ad_cross.sum()
print('구입안함:', n_not, ', 구입함:', n_yes)

n_adA, n_adB = ad_cross.sum(axis=1)
print('광고 A봄:', n_adA, ', 광고 B봄:', n_adB)

구입안함: 900 , 구입함: 100
광고 A봄: 400 , 광고 B봄: 600


광고 A를 보고 상품을 구입하는 경우, 광고에 따라 상품을 구입한 비율이 변하지 않는다면 400명 중 10%, 즉 40명이 상품을 구입한다고 기대할 수 있음. 

광고와 구입이 독립인 변수일 때 기대되는 도수를 `기대도수(expected frequency)` 라고 함. 한편, 실제로 관측된 데이터는 `관측도수 (observed frequency)` 라고 함

모든 셀에서 기대도수의 계산을 수행

In [36]:
ad_ef = pd.DataFrame({'했다': [n_adA * n_yes / n, n_adB * n_yes / n],
                      '하지 않았다' : [n_adA * n_not / n, n_adB * n_not / n]},
                     index = ['A', 'B'])
display(ad_ef)

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


카이제곱검정에서는 `기대도수`와 `관측도수`의 차이를 측정함으로써 검정을 수행

Y 검정통계량은 다음과 같이 구함

$Y = \displaystyle \sum_i \sum_j \dfrac {(O_{ij} - E_{ij})^2} {E_{ij}}$

In [42]:
print('중간값\n', ( (ad_cross - ad_ef)**2 / ad_ef ).sum())

print('\n\n')
y = ( (ad_cross - ad_ef)**2 / ad_ef ).sum().sum()
print('검정통계량 Y:', y)

중간값
 하지 않았다    0.375
했다        3.375
dtype: float64



검정통계량 Y: 3.75


Y는 자유도가 1인 카이제곱분포를 근시적으로 따른다고 알려져 있음.

따르는 분포를 알고 있으면 p 값을 구하는 것은 간단

In [44]:
rv = stats.chi2(1)
p_val = 1 - rv.cdf(y)
print('p-value:', p_val)
print('p-value 가 유의수준 보다 크므로 귀무가설을 채택합니다.')
print('광고 A, B가 제품 구매에 유의한 차이가 인정되지 않는다는 결론')

p-value: 0.052807511416113395
p-value 가 유의수준 보다 크므로 귀무가설을 채택합니다.


`chi2_contingency` 함수를 이용하여 위 계산과정을 간단하게 가능

파라미터로 `교차집계표`를 전달하고, `correction=False`로 함

In [51]:
# correction 이 True이고 자유도가 1이면 연속성에 대한 예이츠 보정을 적용합니다. 
# 보정의 효과는 관찰된 각 값을 해당 예상 값에 대해 0.5씩 조정하는 것입니다.
chi2, p, dof, ef = stats.chi2_contingency(ad_cross, correction=False)

print(f'chi2: {chi2:.4f}, p-value:{p:.4f}, 자유도(df):{dof:.4f}')

print('\n')
print('기대도수(ef)\n')
print(ef)

chi2: 3.7500, p-value:0.0528, 자유도(df):1.0000


기대도수(ef)

[[360.  40.]
 [540.  60.]]
